<a href="https://colab.research.google.com/github/AUT-Student/BigData-HW1/blob/main/BigData_HW1_Q1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libraries

# Download File

In [1]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

id='1-OCBGBtKoY_PadKHcXDyWxHQ2BS8Nulo'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('hw1.zip')

!unzip hw1.zip

Archive:  hw1.zip
replace hw1-files/q3/patches.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: hw1-files/q3/patches.csv  
replace hw1-files/q3/lsh.py? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: hw1-files/q3/lsh.py     
replace hw1-files/q1/dataset1.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: hw1-files/q1/dataset1.txt  
replace hw1-files/q2/games_library.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: hw1-files/q2/games_library.txt  


# PySpart Setting

In [2]:
!pip install pyspark

y
y
y
y
A


In [3]:
!pip install -q findspark

In [16]:
#!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [19]:
#!wget -q https://www-us.apache.org/dist/spark/spark-3.2.1/spark-3.2.1-bin-hadoop2.7.tgz

In [20]:
#!tar xf spark-3.2.1-bin-hadoop2.7.tgz

tar: spark-3.2.1-bin-hadoop2.7.tgz: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now


In [4]:
import findspark
findspark.init()

In [5]:
findspark.find()

'/usr/local/lib/python3.7/dist-packages/pyspark'

In [6]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [7]:
spark

In [8]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels

--2022-03-24 08:46:11--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.202.168.65, 54.237.133.81, 54.161.241.46, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.202.168.65|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.19M  68.2MB/s    in 0.2s    

2022-03-24 08:46:11 (68.2 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13832437/13832437]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
{"tunnels":[{"name":"command_line","uri":"/api/tunnels/command_line","public_url":"https://2906-34-86-174-210.ngrok.io","proto":"https","config":{"addr":"http://localhost:4050","inspect":true},"metrics":{"conns":{"count":0,"gauge":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0},"http":{"count":0,"rate1":0,"rate5":0,"rate15":0,

# Load Dataset

In [15]:
df = spark.read.format("csv").option("sep","\t").option("header", False).load("/content/hw1-files/q1/dataset1.txt")

In [16]:
df

DataFrame[_c0: string, _c1: string]

In [17]:
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)



In [20]:
df.show(5)

+---+--------------------+
|_c0|                 _c1|
+---+--------------------+
|  0|1,2,3,4,5,6,7,8,9...|
|  1|0,5,20,135,2409,8...|
|  2|0,117,135,1220,27...|
|  3|0,12,41,55,1532,1...|
|  4|0,8,14,15,18,27,7...|
+---+--------------------+
only showing top 5 rows



In [21]:
df.count()

49995

In [11]:
!head /content/hw1-files/q1/dataset1.txt -n 20

0	1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94
1	0,5,20,135,2409,8715,8932,10623,12347,12846,13840,13845,14005,20075,21556,22939,23520,28193,29724,29791,29826,30691,31232,31435,32317,32489,34394,35589,35605,35606,35613,35633,35648,35678,38737,43447,44846,44887,49226,49985,623,629,4999,6156,13912,14248,15190,17636,19217,20074,27536,29481,29726,29767,30257,33060,34250,34280,34392,34406,34418,34420,34439,34450,34651,45054,49592
2	0,117,135,1220,2755,12453,24539,24714,41456,45046,49927,6893,13795,16659,32828,41878
3	0,12,41,55,1532,12636,13185,27552,38737
4	0,8,14,15,18,27,72,80,15326,19068,19079,24596,42697,46126,74,77,33269,38792,38822
5	0,1,20,2022,22939,23527,30257,32503,35633,41457,43262,44846,49574,31140,32828
6	0,21,98,2203,3238,5040,8795,9843,9847,15294,1787